In [103]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [104]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0
1,46238094928,17719030306,537418879,535500841,476197532,476197532,280724274,280724274,655300618,427509228
2,36050996554,19021248224,24910995,24910995,0,0,0,0,545300618,518540162
3,29134251061,18614185810,187609472,187609472,194216024,194216024,110492056,110492056,439310873,236874891
4,28856489167,18600847101,297822197,297523644,200404791,200404791,159173790,159173790,362094887,248866239


In [105]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 10
bs_capacity = 20000000000


In [106]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity

In [107]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [108]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [ ]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/10
2000/2000 [==============================] - 17s 8ms/step - loss: 0.0302 - val_loss: 0.0340
Epoch 2/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0265 - val_loss: 0.0333
Epoch 3/10
2000/2000 [==============================] - 13s 7ms/step - loss: 0.0260 - val_loss: 0.0317
Epoch 4/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0257 - val_loss: 0.0330
Epoch 5/10
2000/2000 [==============================] - 12s 6ms/step - loss: 0.0255 - val_loss: 0.0318
Epoch 6/10
2000/2000 [==============================] - 13s 7ms/step - loss: 0.0254 - val_loss: 0.0339
Epoch 7/10
1993/2000 [============================>.] - ETA: 0s - loss: 0.0252

In [101]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[46238094928,537418879,476197532,280724274, 655300618]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[0]) # because I took the input of the first values in x_train for test


[0.9415117  0.00885184 0.01063882 0.00830393 0.0115558 ]
[0.93077304 0.01325895 0.01335097 0.00897079 0.01115676]
[0.93523765 0.00894073 0.01111945 0.00856403 0.01213633]
[0.9410416  0.00593441 0.00492865 0.00433391 0.0119589 ]


In [102]:
# Save trained model
model.save("model.h5")


In [85]:
 [0.9469534  0.00788183 0.00737214 0.00531304 0.00937836]
[0.9410416  0.00593441 0.00492865 0.00433391 0.0119589 ]128 32


SyntaxError: invalid syntax (<ipython-input-85-22dbf48db488>, line 1)